# Análisis de Modelos de Estado para el Metano

## 1. Introducción

Se proporciona una serie de tablas con valores de presión ($P$) y volumen específico ($V$) del metano a distintas temperaturas constantes ($T$). Cada tabla contiene puntos discretos. En el fichero Excel adjunto se encuentran los valores de presión en MPa y volumen en m³/kg a diferentes temperaturas.

Uno de los objetivos es obtener el valor de $V$ dadas unas condiciones donde $P$ y $V$ no son constantes. Para ello, se desea utilizar dos ecuaciones de estado de los gases reales.

## 2. Ecuaciones de Estado de los Gases Reales

### 2.1 Ecuación de Peng-Robinson

La ecuación de Peng-Robinson es una ecuación de estado cúbica, ampliamente utilizada en termodinámica de fluidos para modelar el comportamiento de gases y líquidos. Su forma general es:

$$
P = \frac{RT}{V - b} - \frac{a}{V^2 + 2bV - b^2}
$$

Donde la constante $a$ se calcula como:

$$
a = 0.4572 \cdot \frac{R^2 T_c^2}{P_c} \cdot \alpha(T_r, \omega)
$$

$$
\alpha(T_r, \omega) = 0.4572 \cdot \frac{R^2 T_c^2}{P_c} \cdot \left[1 + k(1 - \sqrt{T/T_c})\right]
$$

$$
k = 0.37 + 1.5\omega - 0.26\omega^2
$$

$$
b = 0.077 \cdot \frac{RT_c}{P_c}
$$

Para el metano, los valores críticos y el factor acéntrico son:

- **Temperatura crítica:** $T_c = 190.56$ K
- **Presión crítica:** $P_c = 4.59$ MPa
- **Volumen crítico:** $V_c = 98.6$ cm³/mol
- **Factor acéntrico:** $\omega = 0.011$

### 2.2 Ecuación del Virial

La ecuación del virial es una expansión en series de la ecuación de estado de un gas real en términos del volumen molar:

$$
Z = \frac{PV}{RT} = 1 + \frac{B(T)}{V} + \frac{C(T)}{V^2} + \dots
$$

Donde $B(T)$ y $C(T)$ son los coeficientes de Virial. El primero se puede aproximar mediante:

$$
B(T) = -0.199 + 0.20 \cdot \exp\left(\frac{-1131}{T^2}\right)
$$

El coeficiente $C(T)$ es muy complejo de obtener experimentalmente.

## 3. Problemas a Resolver

### 3.1 Interpolación y regresión

- Realizar gráficas de interpolación polinomial y regresión de los puntos $PV$ que mejor se ajusten al caso.
- Identificar la mejor curva para resolver el problema.

In [2]:
import pandas as pd
import numpy as np

# leer el excel
df = pd.read_excel('Caso_1.xlsx')


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Supongamos que tidy_df ya está disponible
temperaturas = df["T (K)"].unique()
temperaturas = sorted(temperaturas)

# Función para calcular R^2
def r_squared(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - ss_res/ss_tot

KeyError: 'T (K)'

In [ ]:
polynomial_degrees = [1, 2, 3]

# Almacenaremos resultados en una lista para luego analizarlos
results = []

for T in temperaturas:
    subset = tidy_df[tidy_df["T (K)"] == T]
    P_vals = subset["P (MPa)"].values
    V_vals = subset["V (m³/kg)"].values
    
    # Ordenamos por P ascendente (útil para graficar)
    sort_idx = np.argsort(P_vals)
    P_vals = P_vals[sort_idx]
    V_vals = V_vals[sort_idx]
    
    # Rango de presiones para graficar la curva ajustada
    P_fine = np.linspace(P_vals.min(), P_vals.max(), 100)
    
    fig, ax = plt.subplots(figsize=(6,4))
    ax.scatter(P_vals, V_vals, label="Datos experimentales", color="black", marker="o")
    
    best_r2 = -np.inf
    best_degree = None
    best_fit_curve = None
    
    for deg in polynomial_degrees:
        # Ajuste polinomial: V = a0 + a1*P + a2*P^2 + ...
        coeffs = np.polyfit(P_vals, V_vals, deg)
        # Calculamos predicción sobre los puntos originales (para R^2)
        V_pred = np.polyval(coeffs, P_vals)
        current_r2 = r_squared(V_vals, V_pred)
        
        # Graficamos la curva en P_fine
        V_fine = np.polyval(coeffs, P_fine)
        ax.plot(P_fine, V_fine, label=f"Grado {deg}, R²={current_r2:.4f}")
        
        # Guardamos el mejor ajuste para este T
        if current_r2 > best_r2:
            best_r2 = current_r2
            best_degree = deg
            best_fit_curve = coeffs
    
    ax.set_title(f"T = {T} K: Ajustes polinomiales")
    ax.set_xlabel("Presión (MPa)")
    ax.set_ylabel("Volumen específico (m³/kg)")
    ax.legend()
    plt.tight_layout()
    plt.show()
    
    # Almacenar información del mejor ajuste
    results.append({
        "Temperatura": T,
        "Mejor grado polinomial": best_degree,
        "R^2": best_r2,
        "Coeficientes": best_fit_curve
    })

In [ ]:
for r in results:
    print(f"T = {r['Temperatura']} K")
    print(f"  Mejor grado polinomial: {r['Mejor grado polinomial']}")
    print(f"  R^2 = {r['R^2']:.4f}")
    print(f"  Coeficientes (mayor grado a menor): {r['Coeficientes']}\n")

### 3.2 Interpolación de Hermite

- Representar la función $V = f(T)$ mediante interpolación de Hermite.
- Considerar que la primera derivada $dV/dT = 0.019$ m³/kg·K a una temperatura $T = 300$ K y un volumen específico de $4.1$ m³/kg.

### 3.3 Validación de la Ecuación de Peng-Robinson

- Analizar si la curva interpolada en la sección **3.1** se ajusta a la ecuación de Peng-Robinson.
- Resolver la ecuación para cada punto $P$ y calcular el error cometido.
- Elaborar un diagrama de flujo explicando el procedimiento para resolver esta cuestión.
- Realizar el análisis únicamente para $T = 250$ K con un error mínimo de $0.001$.

### 3.4 Estimación del Coeficiente de Virial

- Utilizando los datos del punto **3.2** y considerando una presión constante de 1 MPa, determinar la temperatura $T$ para la cual el coeficiente de Virial es $6 \times 10^{-4}$ m$^6$/Kg$^2$.
- Calcular el número de iteraciones necesarias para un error de $0.01$, $0.001$ y $10^{-10}$.

### 3.5 Comparación de Estimaciones del Coeficiente de Virial

- Repetir el cálculo del punto **3.4** para los valores $10^{-4}$ m$^6$/Kg$^2$ y $8.5 \times 10^{-4}$ m$^6$/Kg$^2$ con un error de $0.001$.
- Graficar los resultados obtenidos.